In [10]:
'''
    @Author: King
    @Date: 2019.05.16
    @Purpose: 文件数据读写(Python版)
    @Introduction:  spark 实现本地文件读取
    @Datasets: 
    @Link : 
    @Reference : http://dblab.xmu.edu.cn/blog/1708-2/
'''

'\n    @Author: King\n    @Date: 2019.05.16\n    @Purpose: 文件数据读写(Python版)\n    @Introduction:  spark 实现本地文件读取\n    @Datasets: \n    @Link : \n    @Reference : http://dblab.xmu.edu.cn/blog/1708-2/\n'

## Spark2.1.0+入门：文件数据读写(Python版)

![作者](../img/bigdata-roadmap.jpg)
【版权声明】博客内容由厦门大学数据库实验室拥有版权，未经允许，请勿转载！

除了可以对本地文件系统进行读写以外，Spark还支持很多其他常见的文件格式（如文本文件、JSON、SequenceFile等）和文件系统（如HDFS、Amazon S3等）和数据库（如MySQL、HBase、Hive等）。数据库的读写我们将在Spark SQL部分介绍，因此，这里只介绍文件系统的读写和不同文件格式的读写。

### 文件数据读写之前的准备工作

1. 目录下新建一个包含了一些语句的文本文件word.txt（你可以在文本文件中随意输入一些单词，用空格隔开）。

2. 启动 pyspark


In [1]:
from pyspark import SparkContext
sc = SparkContext( 'local', 'test')

### 不同文件格式的文件系统的数据读写

下面分别介绍本地文件系统的数据读写和分布式文件系统HDFS的数据读写。

#### 本地文件文件的数据读写

In [2]:
textFile = sc.textFile("word.txt")

上面代码中，sc.textFile()中的这个textFile是sc的一个方法名称，这个方法用来加载文件数据。这两个textFile不是一个东西，不要混淆。实际上，val后面的是变量textFile，你完全可以换个变量名称，比如,val lines = sc.textFile(“word.txt”)。这里使用相同名称，就是有意强调二者的区别。

注意，要加载本地文件，必须采用“file:///”开头的这种格式。执行上上面这条命令以后，并不会马上显示结果，因为，Spark采用惰性机制，只有遇到“行动”类型的操作，才会从头到尾执行所有操作。所以，下面我们执行一条“行动”类型的语句，就可以看到结果：

In [3]:
textFile.first()

'面 第 一 行 首 先 从 外 部 文 件 d a t a . t x t 中 构 建 得 到 一 个 R D D ，'

first()是一个“行动”（Action）类型的操作，会启动真正的计算过程，从文件中加载数据到变量textFile中，并取出第一行文本。屏幕上会显示很多反馈信息，这里不再给出，你可以从这些结果信息中，找到word.txt文件中的第一行的内容。

练习一下如何把textFile变量中的内容再次写回到另外一个文本文件wordback.txt中：

In [6]:
textFile.saveAsTextFile("writeback")

saveAsTextFile()是一个“行动”（Action）类型的操作，所以，马上会执行真正的计算过程，从word.txt中加载数据到变量textFile中，然后，又把textFile中的数据写回到writeback.txt中。现在我们到/usr/local/spark/mycode/wordcount/目录看一下，会发现，确实多了一个writeback.txt，但是，和我们预期的不一样，它不是一个文件，而是一个文件夹（writeback.txt作为文件夹名称当然是没有问题的，虽然不符合我们平时的习惯）。

#### 本地 JSON 的数据读写

In [7]:
jsonStr = sc.textFile("people.json")

In [8]:
jsonStr.collect()

['{"name":"Michael"}',
 '{"name":"Andy", "age":30}',
 '{"name":"Justin", "age":19}']

从上面执行结果可以看出，people.json文件加载到RDD中以后，在RDD中存在三个字符串。我们下面要做的事情，就是把这三个JSON格式的字符串解析出来，比如说，第一个字符串{“name”:”Michael”}，经过解析后，解析得到key是”name”，value是”Michael”。

现在我们编写程序完成对上面字符串的解析工作。

Scala中有一个自带的JSON库——scala.util.parsing.json.JSON，可以实现对JSON数据的解析。JSON.parseFull(jsonString:String)函数，以一个JSON字符串作为输入并进行解析，如果解析成功则返回一个Some(map: Map[String, Any])，如果解析失败则返回None。

因此，我们可以使用模式匹配来处理解析结果

请执行以下命令：

In [9]:
import json

inputFile =  "people.json"
jsonStrs = sc.textFile(inputFile)
result = jsonStrs.map(lambda s : json.loads(s))
result.collect()

[{'name': 'Michael'},
 {'name': 'Andy', 'age': 30},
 {'name': 'Justin', 'age': 19}]